In [11]:
import pandas as pd
import numpy as np
categories_df = pd.read_csv('../data/amazon_categories.csv')
products_df = pd.read_csv('../data/amazon_products.csv')
data = pd.merge(categories_df, products_df, left_on='id', right_on='category_id')
data = data.drop('id', axis=1)

In [12]:
data.head(10)

,category_name,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,Beading & Jewelry Making,B08YNTXFJG,Duufin 120 Pieces Acrylic Keychain Blanks Set ...,https://m.media-amazon.com/images/I/71czFH2QUM...,https://www.amazon.com/dp/B08YNTXFJG,4.7,510,9.99,0.00,1,False,50
1,Beading & Jewelry Making,B08DXSF5PZ,"DIY Tray Silicone Resin Mold, Oval Epoxy Resin...",https://m.media-amazon.com/images/I/71ZnrCHA49...,https://www.amazon.com/dp/B08DXSF5PZ,4.6,584,8.99,0.00,1,False,0
2,Beading & Jewelry Making,B096JGXQN5,"GTOTd I Love Rock and Roll Pins Set(18 Pack,2....",https://m.media-amazon.com/images/I/71SOJXp-wJ...,https://www.amazon.com/dp/B096JGXQN5,4.5,245,12.79,0.00,1,False,50
3,Beading & Jewelry Making,B0BZXW1ZGS,Xuhal 100 Pack Plastic Ring Sizer Bulk Reusabl...,https://m.media-amazon.com/images/I/81qD7tFV+r...,https://www.amazon.com/dp/B0BZXW1ZGS,5.0,1,16.99,0.00,1,False,0
4,Beading & Jewelry Making,B0BR8B472F,"Large Dice Resin Molds, 2 Styles Silicone Dice...",https://m.media-amazon.com/images/I/71uDVuhf71...,https://www.amazon.com/dp/B0BR8B472F,4.4,10,15.99,0.00,1,False,50
5,Beading & Jewelry Making,B096HTDZR5,10 Pieces 25 mm Bezel Tray Blank Bangle Bracel...,https://m.media-amazon.com/images/I/71M5IJaWA2...,https://www.amazon.com/dp/B096HTDZR5,4.4,130,14.99,0.00,1,False,0
6,Beading & Jewelry Making,B07SVCV8VB,100pcs Adabele Authentic 925 Sterling Silver C...,https://m.media-amazon.com/images/I/51wAAOtlbz...,https://www.amazon.com/dp/B07SVCV8VB,4.5,82,10.49,0.00,1,False,0
7,Beading & Jewelry Making,B0B4689HLD,90 Pcs Antique Silver Jesus Mary Religious Ben...,https://m.media-amazon.com/images/I/713kgDwf7n...,https://www.amazon.com/dp/B0B4689HLD,4.6,65,8.99,0.00,1,False,0
8,Beading & Jewelry Making,B09YRKCH71,Cinvo 40 Pcs Acrylic Pins Bulk Decoration Cute...,https://m.media-amazon.com/images/I/712QiTKefS...,https://www.amazon.com/dp/B09YRKCH71,4.4,61,9.99,0.00,1,False,50
9,Beading & Jewelry Making,B08ZXBT6RY,HUSUM 10 Pieces 6 Inches Curved Beading Needle...,https://m.media-amazon.com/images/I/71bfUVKYJq...,https://www.amazon.com/dp/B08ZXBT6RY,4.2,318,8.99,9.99,1,False,0


In [13]:
pd.options.display.max_rows = 4000
print(data['category_id'].value_counts())

category_id
91     28619
84     24660
270    20846
114    19822
118    18994
95     18514
110    18258
112    17679
116    17393
123    17005
105    16160
122    15626
173    15437
120    13921
113    12839
97     10938
88     10868
107     9580
51      9535
228     9485
72      9480
18      9418
16      9396
141     9364
10      9346
206     9254
71      9242
22      9238
143     9085
230     9020
179     8950
73      8859
43      8858
69      8823
168     8810
139     8777
147     8723
47      8669
12      8664
19      8659
142     8646
13      8611
163     8600
24      8536
174     8527
167     8523
209     8497
28      8487
64      8466
23      8444
63      8443
48      8402
3       8386
57      8364
8       8355
164     8337
180     8325
219     8313
21      8289
175     8283
60      8250
1       8246
6       8198
17      8195
165     8189
153     8146
68      8089
49      8074
138     8066
217     7973
154     7928
210     7854
140     7839
203     7794
223     7789
11      7778


In [103]:
category_id_to_keep_str = ['45','46','47','48','49','50','52','71','72','84','90','91','97','101','103','104','105','107','108','109','110','111','112','113','114','116','118','120','121','122','123','173','174', '270']
category_id_to_keep_str = ['46']
category_id_to_keep = [int(id) for id in category_id_to_keep_str]
filtered_data = data[data['category_id'].isin(category_id_to_keep)]
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 882 entries, 254577 to 255458
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   category_name      882 non-null    object 
 1   asin               882 non-null    object 
 2   title              882 non-null    object 
 3   imgUrl             882 non-null    object 
 4   productURL         882 non-null    object 
 5   stars              882 non-null    float64
 6   reviews            882 non-null    int64  
 7   price              882 non-null    float64
 8   listPrice          882 non-null    float64
 9   category_id        882 non-null    int64  
 10  isBestSeller       882 non-null    bool   
 11  boughtInLastMonth  882 non-null    int64  
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 83.5+ KB


In [69]:
filtered_data.head(20)

,category_name,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
255459,Hair Care Products,B07BHCB9Y2,PURA D'OR Anti-Thinning Biotin Shampoo and Con...,https://m.media-amazon.com/images/I/81NPZAy3Mp...,https://www.amazon.com/dp/B07BHCB9Y2,4.3,0,39.99,0.00,47,True,20000
255460,Hair Care Products,B0BMQ13HGW,"Rosemary Essential Oil for Hair Growth, 100% P...",https://m.media-amazon.com/images/I/81MAIVxzNV...,https://www.amazon.com/dp/B0BMQ13HGW,4.5,0,7.99,15.99,47,False,6000
255461,Hair Care Products,B001VGA4P8,"Pro Tools Detangler Comb, Wide Tooth Comb Deta...",https://m.media-amazon.com/images/I/31Qyw8MkJ-...,https://www.amazon.com/dp/B001VGA4P8,4.7,0,5.00,0.00,47,False,10000
255462,Hair Care Products,B09ST1NL17,26 Inch HD Lace Front Wigs Human Hair Body Wav...,https://m.media-amazon.com/images/I/71NSWwzqe6...,https://www.amazon.com/dp/B09ST1NL17,3.9,0,24.49,0.00,47,False,6000
255463,Hair Care Products,B00E9SP888,"Goody Slideproof Womens Bobby Pin-48 Count,Cri...",https://m.media-amazon.com/images/I/71ngK6KWiP...,https://www.amazon.com/dp/B00E9SP888,4.5,0,2.69,3.59,47,True,9000
255464,Hair Care Products,B0BPR45NX4,Mielle Organics Rosemary Mint Strengthening Le...,https://m.media-amazon.com/images/I/61XcZcOICM...,https://www.amazon.com/dp/B0BPR45NX4,4.5,0,9.49,12.99,47,False,10000
255465,Hair Care Products,B076S6KMY9,OGX Extra Strength Refreshing Scalp + Teatree ...,https://m.media-amazon.com/images/I/71ZdAOm4MF...,https://www.amazon.com/dp/B076S6KMY9,4.5,0,7.00,10.69,47,False,9000
255466,Hair Care Products,B0009V1YR8,CHI Original Ceramic Hair Straightening Flat I...,https://m.media-amazon.com/images/I/41KtWacjX6...,https://www.amazon.com/dp/B0009V1YR8,4.4,0,52.98,99.98,47,False,9000
255467,Hair Care Products,B06Y2DT2GJ,Argan Oil Shampoo and Conditioner Set - Moistu...,https://m.media-amazon.com/images/I/71-y3IjAPS...,https://www.amazon.com/dp/B06Y2DT2GJ,4.5,0,24.99,29.99,47,False,8000
255468,Hair Care Products,B000E8PG98,"Conair Hair Dryer with Retractable Cord, 1875W...",https://m.media-amazon.com/images/I/7164oZSI-i...,https://www.amazon.com/dp/B000E8PG98,4.4,0,18.83,26.99,47,False,9000


In [104]:
print(filtered_data['boughtInLastMonth'].value_counts())

boughtInLastMonth
0        659
50        68
100       50
200       23
300       19
400       16
500       11
1000      10
600        7
700        5
800        3
900        2
3000       2
5000       2
20000      1
10000      1
6000       1
4000       1
2000       1
Name: count, dtype: int64


In [105]:
filtered_data = filtered_data[~filtered_data['boughtInLastMonth'].isin([0])]


In [106]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 223 entries, 254577 to 255410
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   category_name      223 non-null    object 
 1   asin               223 non-null    object 
 2   title              223 non-null    object 
 3   imgUrl             223 non-null    object 
 4   productURL         223 non-null    object 
 5   stars              223 non-null    float64
 6   reviews            223 non-null    int64  
 7   price              223 non-null    float64
 8   listPrice          223 non-null    float64
 9   category_id        223 non-null    int64  
 10  isBestSeller       223 non-null    bool   
 11  boughtInLastMonth  223 non-null    int64  
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 21.1+ KB


In [19]:
filtered_data.head(20)

,category_name,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
566498,Suitcases,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000
566499,Suitcases,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000
566500,Suitcases,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.6,0,365.49,429.99,104,False,300
566501,Suitcases,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.6,0,291.59,354.37,104,False,400
566502,Suitcases,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.5,0,174.99,309.99,104,False,400
566503,Suitcases,B07XSCD2R4,Maxlite 5 Softside Expandable Luggage with 4 S...,https://m.media-amazon.com/images/I/61LnBNsSBS...,https://www.amazon.com/dp/B07XSCD2R4,4.5,0,144.49,0.00,104,False,500
566504,Suitcases,B07MXF4G8K,"Hard Shell Carry on Luggage Airline Approved, ...",https://m.media-amazon.com/images/I/71CghLYrnA...,https://www.amazon.com/dp/B07MXF4G8K,4.5,0,169.99,0.00,104,False,400
566505,Suitcases,B07H515VCZ,"Maxporter II 30"" Hardside Spinner Trunk Luggag...",https://m.media-amazon.com/images/I/81f3h+YHOX...,https://www.amazon.com/dp/B07H515VCZ,4.5,0,299.99,0.00,104,False,100
566506,Suitcases,B08BXBCNMQ,Omni 2 Hardside Expandable Luggage with Spinne...,https://m.media-amazon.com/images/I/91eOWP4myS...,https://www.amazon.com/dp/B08BXBCNMQ,4.5,0,112.63,137.04,104,False,500
566507,Suitcases,B0B9K44XTS,Luggage Sets Expandable Lightweight Suitcases ...,https://m.media-amazon.com/images/I/81dsv5GrCL...,https://www.amazon.com/dp/B0B9K44XTS,4.4,0,209.99,0.00,104,False,200


In [95]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import json

load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

def create_summary(name, content):
    prompt = f"Create a simple but correct summary of an online shop product description based on the following details in less than 500 characters. Remove rating and pricing information and keep color, size and relevant configuration. If details are available describe how the product can be used and who the ideal customer for this product is. Do not add or use information that is not part of the prompt. Your description should be in english. This is the name of the product: {name} This are the details: {content}"

    response = client.chat.completions.create(
        model = model_name, 
        temperature=1.0,
        messages = [{"role" : "assistant", "content" : prompt}],
    )
    return response.choices[0].message.content


In [96]:
import requests
import json

def web_search_basic(
    name, subscription_key, auth_header_name="Ocp-Apim-Subscription-Key", mkt="en-us"    
):
    """Bing Web Search Basic REST call

    This sample makes a call to the Bing Web Search API with a text query and returns relevant pages
    Documentation: https://docs.microsoft.com/en-us/bing/search-apis/bing-web-search/overview

    May throw HTTPError in case of invalid parameters or a server error.

    Args:
        subscription_key (str): Azure subscription key of Bing Web Search service
        auth_header_name (str): Name of the authorization header
        query (str): Query to search for
        mkt (str): Market to search in
    """
    # Construct a request
    endpoint = "https://api.bing.microsoft.com/v7.0/search"
    params = {"q": "site:amazon.de what is the description of " + name, "mkt": mkt}
    headers = {auth_header_name: subscription_key}

    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()

        json_response = response.json()

        description = json_response["webPages"]["value"][0]["snippet"]

        for item in json_response["webPages"]["value"]:           
            description = item["snippet"] + " "

        summary = create_summary(name, description)
        summary = summary.replace("\n", "")
        return summary
    except Exception as ex:
        print(ex)
        print("++The above exception was thrown and handled succesfully++")
        return response


In [107]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

print(len(filtered_data))

limit = 300
counter = 0
# loop through the filtered data and search for the description of each product
for ind in filtered_data.head(limit).index:
    name = filtered_data['title'][ind]

    # print(name)

    description = web_search_basic(name, subscription_key)

    filtered_data.at[ind, 'description'] = description
    counter = counter + 1
    update_progress(counter / limit)


print("Processing complete!")

filtered_data.to_csv('../data/filtered_dataset.csv', index=False)

Progress: [###############-----] 74.3%
Processing complete!


In [82]:
filtered_data.to_csv('../data/filtered_dataset.csv', index=False)